In [ ]:
### FOR COLAB ONLY ###

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive', force_remount=True)

In [ ]:
#cd /content/drive/My Drive

In [ ]:
#!pip install musdb

In [ ]:
### IMPORTS ###

In [ ]:
import musdb
import os
import torch
import torch.optim as optim
import torch.nn as nn
from moviepy.editor import AudioFileClip, ColorClip
from scipy.io.wavfile import write
from modules.model import ssnet
import pickle
from tqdm.notebook import tqdm
import numpy as np

In [ ]:
### BUILD DATASET ###

# mus.audio : The mixture
# mus.stems[0] : The mixture
# mus.stems[1] : The drums
# mus.stems[2] : The bass
# mus.stems[3] : Rest of the accompaniment
# mus.stems[4] : The vocals

# #TODO: Split mus into train and test subsets: 
# --> mus_train = musdb.DB(subsets="train") 
# --> mus_test = musdb.DB(subsets="test")

# #TODO: audio with a number of frames > 9000000 crash the notebook... find a way around this

# #TODO: We want the input to be the mixture (stems[1] or mus.audio)
# --> target has to be everything but vocals, thus: Find a way to mix stems[1], stems[2], stems[3]
# --> temporarly here the target is stems[4] (vocals)

# #Note: Soundtracks are too big to be processed all at once by the model, so here we process the soundtrack by chunks of 100000 frames.

In [ ]:
mus = musdb.DB(root="data/musdb18")

In [ ]:
dataset = []
frames = 30000
for t in tqdm(mus):
    audio = t.stems[0]
    audio_len = audio.shape[0]
    if audio_len > 9000000:
      continue
    target = t.stems[4]
    slices = list(range(0, audio_len, frames))
    for i in range(0, slices[-1], frames):
        if slices[-1] - i < frames:
            continue
        x = torch.tensor(audio[i:i+frames]).T.float()
        x = x.view(1,2,-1)
        y = torch.tensor(target[i:i+frames]).T.float()
        y = y.view(1,2,-1)
        dataset.append((x,y))

In [ ]:
pickle.dump(dataset, open('dataset.pkl', 'wb'))
#dataset = pickle.load(open('dataset.pkl', 'rb'))

In [ ]:
### TRAINING ###

In [ ]:
EPOCHS = 100
LR = 0.01
WD = 0

In [ ]:
ssnet_ = ssnet().float()
optimizer = optim.Adam(ssnet_.parameters(),lr=LR, weight_decay=WD)
loss_function = nn.MSELoss(reduction='mean')

In [ ]:
ssnet_.train()
for epoch in range(EPOCHS):
  for i in dataset:
    x,y = i[0], i[1]
    out = ssnet_(x)
    ssnet_.zero_grad()
    loss = loss_function(out, y)
    loss.backward()
    optimizer.step()
    print(loss)

In [ ]:
ssnet_.eval()

In [ ]:
### TEST MODEL ###

# Here we ask the trained model to predict n chunks of frames. 
# We concatenate these predictions and output them in an audio file

In [ ]:
rebuild_track = []
cnt = 0
for i in dataset:
  cnt += 1
  x = i[0]
  out = ssnet_(x).view(-1,2)
  out = out.detach().numpy()
  rebuild_track.append(out)
  if cnt == 100:
    break

In [ ]:
result = np.concatenate(rebuild_track, axis=0)

In [ ]:
#sample_rate = 44100
#wav_file = "output.wav"
#write(wav_file, sample_rate, audio_data)
#audio_clip = AudioFileClip(wav_file)
#video_duration = audio_clip.duration
#video_clip = ColorClip(size=(640, 480), color=(0, 0, 0), duration=video_duration)
#video_clip = video_clip.set_audio(audio_clip)
#video_clip.fps = 24
#mp4_file = "output.mp4"
#video_clip.write_videofile(mp4_file, codec="libx264", audio_codec="aac", fps=24)